<a href="https://colab.research.google.com/github/PINKDDDD/data_cleaning/blob/main/IQR_%2B_Imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics
%matplotlib inline

In [ ]:
#mount Gdrive
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [ ]:


df = pd.read_csv("/content/drive/My Drive/585/train.csv")
df

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,587620,A,B,A,A,A,A,A,A,B,...,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
188314,587624,A,A,A,A,A,B,A,A,A,...,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
188315,587630,A,B,A,A,A,A,A,B,B,...,0.345883,0.370534,0.24564,0.45808,0.47779,0.445614,0.443374,0.339244,0.503888,5762.64
188316,587632,A,B,A,A,A,A,A,A,B,...,0.704364,0.562866,0.34987,0.44767,0.53881,0.863052,0.852865,0.654753,0.721707,1562.87


In [ ]:
cat_cols = df.columns[1:117]

def frequency_encoding(df, columns):
    for col in columns:
        frequency = df[col].value_counts(normalize=True)
        df[col] = df[col].map(frequency)
    return df

df_en = frequency_encoding(df, cat_cols)
df_en

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,0.751654,0.433294,0.945173,0.318201,0.657064,0.699312,0.975711,0.941355,0.399303,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,0.751654,0.433294,0.945173,0.681799,0.657064,0.699312,0.975711,0.941355,0.399303,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,0.751654,0.433294,0.945173,0.681799,0.342936,0.699312,0.975711,0.941355,0.399303,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,0.248346,0.433294,0.945173,0.318201,0.657064,0.699312,0.975711,0.941355,0.399303,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,0.751654,0.433294,0.945173,0.318201,0.657064,0.699312,0.975711,0.941355,0.399303,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188313,587620,0.751654,0.433294,0.945173,0.681799,0.657064,0.699312,0.975711,0.941355,0.399303,...,0.242437,0.289949,0.24564,0.30859,0.32935,0.223038,0.220003,0.333292,0.208216,1198.62
188314,587624,0.751654,0.566706,0.945173,0.681799,0.657064,0.300688,0.975711,0.941355,0.600697,...,0.334270,0.382000,0.63475,0.40455,0.47779,0.307628,0.301921,0.318646,0.305872,1108.34
188315,587630,0.751654,0.433294,0.945173,0.681799,0.657064,0.699312,0.975711,0.058645,0.399303,...,0.345883,0.370534,0.24564,0.45808,0.47779,0.445614,0.443374,0.339244,0.503888,5762.64
188316,587632,0.751654,0.433294,0.945173,0.681799,0.657064,0.699312,0.975711,0.941355,0.399303,...,0.704364,0.562866,0.34987,0.44767,0.53881,0.863052,0.852865,0.654753,0.721707,1562.87


In [ ]:
# detect the outliers --> obtain positions of outliers

num_cols = df_en.columns[117:131]

Q1 = df_en[num_cols].quantile(0.25)
Q3 = df_en[num_cols].quantile(0.75)
IQR = Q3 - Q1

outlier_positions = []
df_handle = df_en.copy()
for col in num_cols:
    lower_bound = Q1[col] - 1.5 * IQR[col]
    upper_bound = Q3[col] + 1.5 * IQR[col]
    outlier_mask = (df_en[col] < lower_bound) | (df_en[col] > upper_bound)
    outlier_indices = df_en[outlier_mask].index.tolist()
    for index in outlier_indices:
      #replace the outlier with None value
        df_handle.loc[index,col] = None
        outlier_positions.append((col, index))
#print(df_handle)
#print(outlier_positions)

            id      cat1      cat2      cat3      cat4      cat5      cat6  \
0            1  0.751654  0.433294  0.945173  0.318201  0.657064  0.699312   
1            2  0.751654  0.433294  0.945173  0.681799  0.657064  0.699312   
2            5  0.751654  0.433294  0.945173  0.681799  0.342936  0.699312   
3           10  0.248346  0.433294  0.945173  0.318201  0.657064  0.699312   
4           11  0.751654  0.433294  0.945173  0.318201  0.657064  0.699312   
...        ...       ...       ...       ...       ...       ...       ...   
188313  587620  0.751654  0.433294  0.945173  0.681799  0.657064  0.699312   
188314  587624  0.751654  0.566706  0.945173  0.681799  0.657064  0.300688   
188315  587630  0.751654  0.433294  0.945173  0.681799  0.657064  0.699312   
188316  587632  0.751654  0.433294  0.945173  0.681799  0.657064  0.699312   
188317  587633  0.248346  0.566706  0.945173  0.318201  0.657064  0.699312   

            cat7      cat8      cat9  ...     cont6     cont7  

In [ ]:
df_train = df_handle.dropna()
# get x and y
y1= df_train[['cont7']].values
y2= df_train[['cont9']].values
y3= df_train[['cont10']].values
x = df_train.drop(['cont7','cont9','cont10'], axis=1)
#get x y train and test
x_train, x_test, y1_train, y1_test = train_test_split(
    x, y1, test_size=0.20, random_state=42)
x_train, x_test, y2_train, y2_test = train_test_split(
    x, y2, test_size=0.20, random_state=42)
x_train, x_test, y3_train, y3_test = train_test_split(
    x, y3, test_size=0.20, random_state=42)
#train model and early stopping
model_c7 = Sequential()
model_c7.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model_c7.add(Dense(25, activation='relu'))
model_c7.add(Dense(1))
model_c7.compile(loss='mean_squared_error', optimizer='adam')


monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5,
        verbose=1, mode='auto', restore_best_weights=True)
model_c7.fit(x_train,y1_train,validation_data=(x_test,y1_test),
        callbacks=[monitor],verbose=2,epochs=1000)

model_c9 = Sequential()
model_c9.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model_c9.add(Dense(25, activation='relu'))
model_c9.add(Dense(1))
model_c9.compile(loss='mean_squared_error', optimizer='adam')


monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5,
        verbose=1, mode='auto', restore_best_weights=True)
model_c9.fit(x_train,y2_train,validation_data=(x_test,y2_test),
        callbacks=[monitor],verbose=2,epochs=1000)

model_c10 = Sequential()
model_c10.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model_c10.add(Dense(25, activation='relu'))
model_c10.add(Dense(1))
model_c10.compile(loss='mean_squared_error', optimizer='adam')


monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5,
        verbose=1, mode='auto', restore_best_weights=True)
model_c10.fit(x_train,y3_train,validation_data=(x_test,y3_test),
        callbacks=[monitor],verbose=2,epochs=1000)
#outlier replacement
nan_rows_c7 = df_handle[df_handle['cont7'].isna()]
nan_rows_c9 = df_handle[df_handle['cont9'].isna()]
nan_rows_c10 = df_handle[df_handle['cont10'].isna()]

x_c7 = nan_rows_c7[x.columns]
x_c9 = nan_rows_c9[x.columns]
x_c10 = nan_rows_c10[x.columns]

y_pred_c7 = model_c7.predict(x_c7)
y_pred_c9 = model_c9.predict(x_c9)
y_pred_c10 = model_c10.predict(x_c10)

df_en.loc[df_handle['cont7'].isna(), 'cont7'] = y_pred_c7
df_en.loc[df_handle['cont9'].isna(), 'cont9'] = y_pred_c9
df_en.loc[df_handle['cont10'].isna(), 'cont10'] = y_pred_c10



#print(df_en)

Epoch 1/1000
4312/4312 - 10s - loss: 11863.5332 - val_loss: 3.6601 - 10s/epoch - 2ms/step
Epoch 2/1000
4312/4312 - 9s - loss: 5239.2739 - val_loss: 37.2799 - 9s/epoch - 2ms/step
Epoch 3/1000
4312/4312 - 8s - loss: 4858.0903 - val_loss: 49.5783 - 8s/epoch - 2ms/step
Epoch 4/1000
4312/4312 - 11s - loss: 3474.8259 - val_loss: 1.3808 - 11s/epoch - 3ms/step
Epoch 5/1000
4312/4312 - 11s - loss: 2817.3433 - val_loss: 2.9521 - 11s/epoch - 3ms/step
Epoch 6/1000
4312/4312 - 15s - loss: 1467.5027 - val_loss: 12.2886 - 15s/epoch - 3ms/step
Epoch 7/1000
4312/4312 - 13s - loss: 1002.9171 - val_loss: 1278.8741 - 13s/epoch - 3ms/step
Epoch 8/1000
4312/4312 - 8s - loss: 990.7964 - val_loss: 95.3498 - 8s/epoch - 2ms/step
Epoch 9/1000
4312/4312 - 11s - loss: 786.6306 - val_loss: 0.2958 - 11s/epoch - 2ms/step
Epoch 10/1000
4312/4312 - 14s - loss: 484.0310 - val_loss: 377.6984 - 14s/epoch - 3ms/step
Epoch 11/1000
4312/4312 - 13s - loss: 554.1675 - val_loss: 0.8890 - 13s/epoch - 3ms/step
Epoch 12/1000
4312/

In [ ]:
# Measure MSE error.
score7 = metrics.mean_squared_error(y_pred_c7,y1_test[1000:3659])
print("Final score 7 (MSE): {}".format(score7))


score9 = metrics.mean_squared_error(y_pred_c9,y2_test[:13194])
print("Final score 9 (MSE): {}".format(score9))


score10 = metrics.mean_squared_error(y_pred_c10,y3_test[:140])
print("Final score 10 (MSE): {}".format(score10))

Final score 7 (MSE): 0.7956338336466586
Final score 9 (MSE): 0.25559813444091983
Final score 10 (MSE): 1.1730475658469328


In [ ]:
# the row that contains at least one outlier

num_cols = df.columns[117:131]

df_outliers = df_en.copy()

Q1 = df[num_cols].quantile(0.25)
Q3 = df[num_cols].quantile(0.75)
IQR = Q3 - Q1

for col in num_cols:
    df_outliers[col + '_outlier'] = ((df_outliers[col] < (Q1[col] - 1.5 * IQR[col])) | (df_outliers[col] > (Q3[col] + 1.5 * IQR[col])))

outlier_rows = df_outliers.loc[(df_outliers[num_cols + '_outlier'].sum(axis=1) > 0), :].index.tolist()

# out_amount = len(outlier_rows)
# out_amount

print(outlier_rows)

[14, 33, 42, 45, 62, 76, 77, 106, 135, 179, 186, 205, 207, 214, 251, 269, 275, 301, 302, 307, 310, 327, 329, 333, 342, 364, 369, 372, 375, 390, 393, 401, 406, 412, 413, 422, 428, 440, 444, 471, 474, 484, 489, 490, 503, 505, 520, 522, 535, 541, 566, 567, 568, 578, 579, 598, 618, 620, 647, 659, 669, 677, 681, 689, 730, 731, 754, 761, 769, 791, 823, 860, 865, 879, 889, 891, 909, 922, 924, 931, 938, 941, 949, 966, 989, 996, 1002, 1006, 1019, 1021, 1031, 1053, 1055, 1061, 1074, 1094, 1110, 1144, 1170, 1189, 1193, 1195, 1198, 1221, 1239, 1244, 1255, 1274, 1285, 1297, 1301, 1307, 1311, 1315, 1322, 1335, 1342, 1343, 1354, 1355, 1363, 1376, 1377, 1389, 1390, 1398, 1411, 1456, 1491, 1507, 1531, 1536, 1543, 1546, 1549, 1568, 1587, 1606, 1607, 1617, 1624, 1632, 1654, 1655, 1666, 1673, 1679, 1681, 1703, 1704, 1719, 1737, 1741, 1773, 1776, 1784, 1795, 1804, 1819, 1822, 1831, 1836, 1841, 1858, 1882, 1890, 1910, 1917, 1924, 1928, 1937, 1941, 1943, 1949, 1963, 1980, 1997, 2019, 2082, 2089, 2100, 2103, 